In [19]:
from PIL import Image
import cv2
import numpy as np
import dlib

In [20]:
cd C:\Users\tarek\OneDrive\Desktop\Attendance

C:\Users\tarek\OneDrive\Desktop\Attendance


In [21]:
# Resnet: https://arxiv.org/abs/1512.03385
face_detector = dlib.get_frontal_face_detector()
points_detector = dlib.shape_predictor('C:/Users/tarek/OneDrive/Desktop/Attendance/Weights/shape_predictor_68_face_landmarks.dat')
face_descriptor_extractor = dlib.face_recognition_model_v1('C:/Users/tarek/OneDrive/Desktop/Attendance/Weights/dlib_face_recognition_resnet_model_v1.dat')

In [22]:
# Load the face_descriptors array
descriptors_path='C:/Users/tarek/OneDrive/Desktop/Attendance/Model/face_descriptors_final.npy'
loaded_descriptors = np.load(descriptors_path)
print(f"Loaded face descriptors with shape: {loaded_descriptors.shape}")


Loaded face descriptors with shape: (45, 128)


In [23]:
# Load the mp dictionary from a file
import pickle
def load_mp(file_path):
    with open(file_path, 'rb') as f:
        loaded_mp = pickle.load(f)
    print(f"Dictionary loaded from {file_path}")
    return loaded_mp

# Load the dictionary
loaded_mp = load_mp('C:/Users/tarek/OneDrive/Desktop/Attendance/Model/mp_final.pkl')
print(loaded_mp)


Dictionary loaded from C:/Users/tarek/OneDrive/Desktop/Attendance/Model/mp_final.pkl
{'C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/train/2020331001/Abhishek3.png': '2020331001', 'C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/train/2020331001/Abhishek4.png': '2020331001', 'C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/train/2020331001/WhatsApp Image 2025-01-29 at 12.48.07 PM.jpeg': '2020331001', 'C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/train/2020331001/WhatsApp Image 2025-01-29 at 12.48.25 PM.jpeg': '2020331001', 'C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/train/2020331001/WhatsApp Image 2025-01-29 at 12.48.44 PM.jpeg': '2020331001', 'C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/train/2020331001/WhatsApp Image 2025-01-29 at 12.48.51 PM.jpeg': '2020331001', 'C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/train/2020331001/WhatsApp Image 2025-01-29 at 12.49.00 PM.jpeg': '2020331001', 'C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/train/202

In [24]:
mp_list = list(loaded_mp.items())

In [25]:
def predict_name(path):
    image=Image.open(path).convert('RGB')
    image_np=np.array(image,'uint8')
    face_detection=face_detector(image_np,1)
    for face in face_detection:
        points=points_detector(image_np,face)

        face_descriptor = face_descriptor_extractor.compute_face_descriptor(image_np, points)
        face_descriptor = [f for f in face_descriptor]
        face_descriptor = np.asarray(face_descriptor, dtype=np.float64)
        face_descriptor = face_descriptor[np.newaxis, :]

        distances = np.linalg.norm(face_descriptor - loaded_descriptors, axis = 1)
        min_index=np.argmin(distances)
        min_distance=distances[min_index]
        print(min_distance)
        if min_distance<=0.5:
            name_pred=mp_list[min_index][1]
        else:
            name_pred='Undefined'

        #cv2.putText(image_np, 'Pred: ' + str(name_pred), (10, 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255))
        #cv2.putText(image_np, 'Exp : ' + str(name_real), (10, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,255,0))
        print(name_pred)

    image_np_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)

    # Display the image with annotations
    #plt.figure(figsize=(6, 6))
    #plt.imshow(image_np_rgb)
    #plt.axis('off')  # Hide axes
    #plt.title(f"Prediction: {name_pred} | Expected: {name_real}")
    #plt.show()
    return name_pred



In [26]:
np.linalg.norm(loaded_descriptors[31] - loaded_descriptors[31])

0.0

In [27]:

test_image_path='C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/2020331023/3.jpg'
predict_name(test_image_path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\tarek\\OneDrive\\Desktop\\Attendance\\cropped\\2020331023\\3.jpg'

In [28]:
from sklearn.metrics import accuracy_score
import os

true_labels = []
predicted_labels = []

test_path='C:/Users/tarek/OneDrive/Desktop/Attendance/cropped/test'


In [29]:
for class_idx, class_name in enumerate(os.listdir(test_path)):
    class_dir = os.path.join(test_path, class_name)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            true_labels.append(class_name)
            predicted_label = predict_name(img_path)
            predicted_labels.append(predicted_label)
            print(f"True Label: {class_name}, Predicted Label: {predicted_label}")
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

0.20190363886369805
2020331001
True Label: 2020331001, Predicted Label: 2020331001
0.27370631794152117
2020331001
True Label: 2020331001, Predicted Label: 2020331001
0.285887820737966
2020331017
True Label: 2020331017, Predicted Label: 2020331017
0.2783898143962235
2020331023
True Label: 2020331023, Predicted Label: 2020331023
0.2881904857723801
2020331023
True Label: 2020331023, Predicted Label: 2020331023
0.22720809994219537
2020331023
True Label: 2020331023, Predicted Label: 2020331023
0.18967203977171337
2020331048
True Label: 2020331048, Predicted Label: 2020331048
0.28761681607553435
2020331048
True Label: 2020331048, Predicted Label: 2020331048
0.22966963733154827
2020331048
True Label: 2020331048, Predicted Label: 2020331048
0.30683223501162343
2020331067
True Label: 2020331067, Predicted Label: 2020331067
0.36590104607438795
2020331067
True Label: 2020331067, Predicted Label: 2020331067
0.2792676108839262
2020331097
True Label: 2020331097, Predicted Label: 2020331097
Test Accu